## 00. 환경설정

In [1]:
import os
import pandas as pd
import seaborn as sns
import numpy as np
import random
from matplotlib import pyplot as plt
from datetime import datetime
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

## 01. 추천시스템을 만들고 평가해보자
- 데이터를 train/test로 나눈다
- train 데이터를 이용해 추천시스템을 만들고 test 데이터로 평가한다
- 평가 지표에 따라 **`1) 추천 결과의 형태`**가 달라지거나, **`2) train/test 데이터를 나누는 방법`**이 달라진다. 

### 무비렌즈 데이터 불러오기

In [2]:
path = "../data/ml-100k/"
ratings_df = pd.read_csv(path + 'u.data', sep='\t', encoding='latin-1', header=None)
ratings_df.columns = ['user_id', 'movie_id', 'rating', 'timestamp']
users_df = pd.read_csv(path + 'u.user', sep='|', encoding='latin-1', header=None)
users_df.columns = ['user_id', 'age', 'gender', 'occupation', 'zip_code']

In [3]:
print(ratings_df.shape)

(100000, 4)


In [4]:
ratings_df.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


### Top K Recommendation
- 사용자 혹은 세션별로 Top K 추천 결과를 생성하여 성능을 평가한다
- 현재 주어진 무비렌즈 데이터의 경우 사용자의 영화에 대한 평점 데이터로 이루어져있다.
- 평가 지표를 구하기 위해서는 사용자에게 영화가 추천되었을 때, 사용자가 이 영화에 관련이 있는지 아닌지에 대한 기준이 필요하다.
- **`따라서 사용자가 영화를 선호한다는 정의는 4.0점 이상의 평가를 내린 것으로 가정한다.`**
- Top K 추천 결과 생성은 Steam 로직을 사용한 평균 평점 기반 추천 로직을 사용하고 이를 통해 지표를 직접 구해본다.

### 추천시스템 성능 평가를 위한 test data와 predict data 생성

#### 데이터를 train/test로 나눈다.

In [5]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(ratings_df, test_size=0.2, random_state=10)

In [6]:
print(train_df.shape)
print(test_df.shape)

(80000, 4)
(20000, 4)


#### 평균 평점 기반 추천을 위한 영화별 통계량 계산

In [7]:
movie_statistics = pd.DataFrame({
    'numUsers' : train_df.groupby('movie_id')['user_id'].nunique(),
    'avgRating' : train_df.groupby('movie_id')['rating'].mean(),
    'stdRating' : train_df.groupby('movie_id')['rating'].std()
}).reset_index()

movie_statistics.head()

,movie_id,numUsers,avgRating,stdRating
0,1,347,3.896254,0.912763
1,2,112,3.205357,0.950471
2,3,72,3.083333,1.207372
3,4,168,3.517857,1.002830
4,5,65,3.323077,1.001681


#### Stem rating 구하기

In [8]:
movie_statistics['steamRating'] = movie_statistics['avgRating'] - (movie_statistics['avgRating'] - 3.0) \
                                  * np.power(2, -np.log10(movie_statistics['numUsers']))
topk_df = movie_statistics.sort_values(by = 'steamRating', ascending=False)
topk_df

,movie_id,numUsers,avgRating,stdRating,steamRating
317,318,246,4.487805,0.821803,4.204142
63,64,231,4.445887,0.771992,4.164946
482,483,200,4.450000,0.721180,4.155769
407,408,90,4.533333,0.673745,4.137647
49,50,461,4.347072,0.897422,4.134485
...,...,...,...,...,...
436,437,5,1.000000,0.000000,2.232024
313,314,5,1.000000,0.000000,2.232024
367,368,27,1.740741,0.944319,2.207649
456,457,22,1.590909,1.007547,2.146595


### Precision / Recall / MAP

In [9]:
def get_precision(relevant, recommend):
    _intersection = set(recommend).intersection(set(relevant))
    return len(_intersection) / len(recommend)

def get_recall(relevant, recommend):
    _intersection = set(recommend).intersection(set(relevant))
    return len(_intersection) / len(relevant)

def get_average_precision(relevant, recommend):
    _precisions = []
    
    for i in range(len(recommend)):
        _recommend = recommend[:i+1]
        _precisions.append(get_precision(relevant, _recommend))
    
    return np.mean(_precisions)

#### 개별 사용자에 대해서 추천을 수행하고 각 지표를 구한 뒤에 이를 합친다.

In [10]:
test_user_set = set(test_df['user_id'].unique())

k = 10

recommend_item = topk_df['movie_id'][:k].tolist()

precisions = []
recalls = []
average_precisions = []

for user_id in list(test_user_set):
    test_user_rating_df = test_df[(test_df['user_id'] == user_id) & (test_df['rating'] >= 4.0)]
    relevant_item = test_user_rating_df.sort_values(by='rating', ascending=False)['movie_id'].tolist()
    
    # 테스트 데이터에 있는 유저 가운데 선호 영화가 아예 없는 케이스도 존재
    if len(relevant_item) == 0:
        continue
        
    # Precision @ K
    precision = get_precision(relevant_item, recommend_item)
    precisions.append(precision)
    
    # Recall @ K
    recall = get_recall(relevant_item, recommend_item)
    recalls.append(recall)
    
    # MAP @ K
    average_precision = get_average_precision(relevant_item, recommend_item)
    average_precisions.append(average_precision)
    
print("Precision @ K :", round(np.mean(precisions), 3))
print("Recall    @ K :", round(np.mean(recalls), 3))
print("MAP       @ K :", round(np.mean(average_precisions), 3))

Precision @ K : 0.055
Recall    @ K : 0.047
MAP       @ K : 0.051


In [11]:
print(len(test_user_set))
print(len(precisions))

943
926


### NDCG(Normalized Discounted Cumulative Gain)

#### 특정 사용자에 대해서 5개의 아이템이 추천되었을 때, cumulative gain 구하기

In [12]:
relevant_item = ['a', 'b', 'c']
recommend_item = ['d', 'c', 'a', 'b', 'e']
k = len(recommend_item)

cg = []
for item in recommend_item:
    if item in relevant_item:
        cg.append(1)
    else:
        cg.append(0)
        
cg

[0, 1, 1, 1, 0]

#### k에 대한 discount 구하기

In [13]:
discount = np.log2(np.arange(k) + 2)
discount

array([1.        , 1.5849625 , 2.        , 2.32192809, 2.5849625 ])

#### Discounted cg 구하기

In [14]:
dcg = np.sum(np.divide(cg, discount))
dcg

1.5616063116448506

#### Ideal cumulative gain 구하기

- 유저의 relevant item 개수가 k보다 적을 경우가 존재한다. 예를 들어
    1) k=5, 유저의 relevant item 개수가 5개 이상이라면
        idcg = (1/log2) + (1/log3) + (1/log4) + (1/log5) + (1/log6)이지만,
    2) k = 5, 유저의 relevant item 개수가 3개라면
        idcg = (1/log2) + (1/log3) + (1/log4)이다.

In [15]:
k_for_icg = min(k, len(relevant_item))
icg = np.zeros(k)

for i in range(k_for_icg):
    icg[i] += 1

icg

array([1., 1., 1., 0., 0.])

### Ideal DCG 구하기

In [16]:
idcg = np.sum(np.divide(icg, discount))
idcg

2.1309297535714578

In [17]:
ndcg = dcg / idcg
ndcg

0.7328286204777911

#### K개의 아이템이 추천되었을 때 NDCG 값을 구한다.

In [18]:
def get_ndcg(relevant_item, recommend_item):
    k = len(recommend_item)
    discount = np.log2(np.arange(k) + 2)
    cg = []
    
    for item in recommend_item:
        if item in relevant_item:
            cg.append(1)
        else:
            cg.append(0)
            
    dcg = np.sum(np.divide(cg, discount))
    
    k_for_icg = min(k, len(relevant_item))
    icg = np.zeros(k)
    for i in range(k_for_icg):
        icg[i] += 1
        
    idcg = np.sum(np.divide(icg, discount))
    
    return dcg / idcg

In [19]:
relevant_item = ['a', 'b', 'd', 'f']
recommend_item = ['d', 'c', 'a', 'b', 'e']
get_ndcg(relevant_item, recommend_item)

0.75369761125927

#### 개별 사용자에 대해서 추천을 수행하고 NDCG 지표를 구한 뒤에 이를 합친다.

In [20]:
k = 10
recommend_item = topk_df['movie_id'][:k].tolist()
ndcgs = []

for user_id in list(test_user_set):
    test_user_rating_df = test_df[(test_df['user_id']==user_id) & (test_df['rating'] >= 4.0)]
    relevant_item = test_user_rating_df.sort_values(by='rating', ascending=False)['movie_id'].tolist()
    
    # 테스트 데이터에 있는 유저 가운데 선호 영화가 아예 없는 케이스도 존재함. (4.0 이상)
    if len(relevant_item) == 0:
        continue
        
    # NDCG @ K
    ndcg = get_ndcg(relevant_item, recommend_item)
    ndcgs.append(ndcg)
    
print("NDCG @ K:", round(np.mean(ndcgs), 3))

NDCG @ K: 0.059


### Rating Prediction
- Top K Recommendation이 아니라 사용자가 아이템에 대해 내린 선호도를 정확히 예측하는 문제로 접근해보자.
- 3가지 간단한 추천 알고리즘을 사용해 RMSE, MAE 값을 구해보자
    - 영화 평균 평점으로 예측
    - 사용자 평균 평점으로 예측

In [21]:
# train / test 데이터에 있는 사용자와 영화 리스트를 잠시 살펴보자.

# 사용자
train_user_set = set(train_df['user_id'].unique())
test_user_set = set(test_df['user_id'].unique())
print("test 데이터에 존재하지만 train 데이터에 등장하지 않는 유저: ", len(test_user_set - train_user_set))

# 영화
train_movie_set = set(train_df['movie_id'].unique())
test_movie_set = set(test_df['movie_id'].unique())
print("test 데이터에 존재하지만 train 데이터에 등장하지 않는 영화: ", len(test_movie_set - train_movie_set))

test 데이터에 존재하지만 train 데이터에 등장하지 않는 유저:  0
test 데이터에 존재하지만 train 데이터에 등장하지 않는 영화:  41


#### 영화 평균 평점 구하기
- train 데이터에 있는 평점을 사용해 영화 평균 평점을 구한다
- test 데이터의 평점을 예측할 때 이 평균 평점을 사용한다
- 만약 test 데이터에 등장한 영화에 대한 평균 평점이 train 데이터에 없는 경우에는 전체 평균 평점으로 예측한다.

In [22]:
train_movie_df = train_df.groupby('movie_id').mean()
train_movie_df

,user_id,rating,timestamp
movie_id,,,
1,472.008646,3.896254,8.829567e+08
2,492.446429,3.205357,8.833241e+08
3,469.527778,3.083333,8.821284e+08
4,478.779762,3.517857,8.826822e+08
5,421.646154,3.323077,8.826885e+08
...,...,...,...
1676,851.000000,2.000000,8.757317e+08
1677,854.000000,3.000000,8.828144e+08
1679,863.000000,3.000000,8.892895e+08


In [23]:
from collections import defaultdict

movie_rating = defaultdict(float)
for index, row in train_movie_df.iterrows():
    movie_rating[index] = row.rating
    
movie_rating

defaultdict(float,
            {1: 3.8962536023054755,
             2: 3.205357142857143,
             3: 3.0833333333333335,
             4: 3.517857142857143,
             5: 3.3230769230769233,
             6: 3.6666666666666665,
             7: 3.7774294670846396,
             8: 3.947674418604651,
             9: 3.904382470119522,
             10: 3.802469135802469,
             11: 3.849740932642487,
             12: 4.3875598086124405,
             13: 3.4149659863945576,
             14: 4.006993006993007,
             15: 3.7711864406779663,
             16: 3.125,
             17: 3.142857142857143,
             18: 2.75,
             19: 3.9791666666666665,
             20: 3.4107142857142856,
             21: 2.7857142857142856,
             22: 4.138211382113822,
             23: 4.1644736842105265,
             24: 3.3970588235294117,
             25: 3.4463519313304722,
             26: 3.459016393442623,
             27: 2.9782608695652173,
             28: 3.944206008

In [24]:
total_average_rating = train_df['rating'].mean()
print("전체 평균:", total_average_rating)

전체 평균: 3.528825


#### 평점 예측하기

In [25]:
test_df['predicted_rating_movie'] = test_df['movie_id'].apply(lambda x: movie_rating.get(x, total_average_rating))
test_df

,user_id,movie_id,rating,timestamp,predicted_rating_movie
33226,463,50,4,890530818,4.347072
64804,277,278,1,879543879,3.270833
39763,221,215,4,875245514,3.712575
51270,748,71,3,879454546,3.785714
9698,169,260,1,891269104,2.553398
...,...,...,...,...,...
84466,216,1067,5,881432392,3.433333
39137,130,342,3,881076199,2.952381
3153,270,306,5,876953744,3.921053
80052,850,705,5,883195034,3.962264


#### 사용자 평균 평점 구하기
- train 데이터에 있는 평점을 사용해 사용자별 평균 평점을 구한다
- test 데이터의 평점을 예측할 떄 해당 사용자의 평균 평점을 사용한다
- 마찬가지로 test 데이터에 등장한 사용자에 대한 평균 평점이 train 데이터에 없는 경우에는 전체 평균 평점으로 예측한다.

In [26]:
train_user_df = train_df.groupby('user_id').mean()
train_user_df

,movie_id,rating,timestamp
user_id,,,
1,136.190265,3.544248,8.773953e+08
2,255.620000,3.720000,8.885941e+08
3,318.682927,2.829268,8.892372e+08
4,289.272727,4.272727,8.920027e+08
5,290.736486,2.986486,8.761934e+08
...,...,...,...
939,523.666667,4.222222,8.802613e+08
940,347.280899,3.415730,8.858824e+08
941,387.941176,4.058824,8.750489e+08


In [27]:
from collections import defaultdict

user_rating = defaultdict(float)
for index, row in train_user_df.iterrows():
    user_rating[index] = row.rating
    
user_rating

defaultdict(float,
            {1: 3.5442477876106193,
             2: 3.72,
             3: 2.8292682926829267,
             4: 4.2727272727272725,
             5: 2.9864864864864864,
             6: 3.66120218579235,
             7: 3.9571865443425076,
             8: 3.7058823529411766,
             9: 4.5,
             10: 4.23448275862069,
             11: 3.4726027397260273,
             12: 4.357142857142857,
             13: 3.1130268199233715,
             14: 4.090909090909091,
             15: 2.8181818181818183,
             16: 4.2272727272727275,
             17: 2.909090909090909,
             18: 3.8727272727272726,
             19: 3.6470588235294117,
             20: 3.121212121212121,
             21: 2.5531914893617023,
             22: 3.3679245283018866,
             23: 3.6129032258064515,
             24: 4.2745098039215685,
             25: 4.0606060606060606,
             26: 2.9444444444444446,
             27: 3.15,
             28: 3.75,
             29: 3.

In [28]:
# 평점 예측하기
test_df['predicted_rating_user'] = test_df['user_id'].apply(lambda x: user_rating.get(x, total_average_rating))
test_df

,user_id,movie_id,rating,timestamp,predicted_rating_movie,predicted_rating_user
33226,463,50,4,890530818,4.347072,2.805556
64804,277,278,1,879543879,3.270833,3.534884
39763,221,215,4,875245514,3.712575,3.644628
51270,748,71,3,879454546,3.785714,3.750000
9698,169,260,1,891269104,2.553398,4.033333
...,...,...,...,...,...,...
84466,216,1067,5,881432392,3.433333,3.774775
39137,130,342,3,881076199,2.952381,4.018382
3153,270,306,5,876953744,3.921053,4.327586
80052,850,705,5,883195034,3.962264,4.473684


In [29]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 영화 평균 평점 사용
mse = mean_squared_error(test_df['rating'].values, test_df['predicted_rating_movie'].values)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test_df['rating'].values, test_df['predicted_rating_movie'].values)

print("영화 평균 평점 사용 예측:", round(mae, 3), round(rmse, 3))

# 사용자 평균 평점 사용
mse = mean_squared_error(test_df['rating'].values, test_df['predicted_rating_user'].values)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test_df['rating'].values, test_df['predicted_rating_user'].values)

print("사용자 평균 평점 사용 예측:", round(mae, 3), round(rmse, 3))

영화 평균 평점 사용 예측: 0.819 1.031
사용자 평균 평점 사용 예측: 0.837 1.047
